# Red Wine Quality

Kaggle link: https://www.kaggle.com/uciml/red-wine-quality-cortez-et-al-2009

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import wandb

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

First, we need to import Pytorch

In [ ]:
import torch
from torch import nn
from torch import optim
from torch.utils import data
import wandb
wandb.init(project="simple_regression")

# Data Processing

In [6]:
train_data = pd.read_csv('/kaggle/input/red-wine-quality-cortez-et-al-2009/winequality-red.csv')
train_data.head()

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/red-wine-quality-cortez-et-al-2009/winequality-red.csv'

We need to separate features from target

In [ ]:
n_train = train_data.shape[0]
all_features = train_data.iloc[:, 1:-1]
all_features = all_features.apply(lambda x: (x - x.mean()) / (x.std()))
train_features = torch.tensor(all_features[:n_train].values, dtype=torch.float32)
train_features

In [ ]:
trains_labels = train_data.quality.values.reshape(-1, 1)
trains_labels

In [ ]:
trains_mean = trains_labels.mean()
trains_std = trains_labels.std()
trains_labels = (trains_labels - trains_mean) / trains_std
train_labels = torch.tensor(trains_labels,
                            dtype=torch.float32)
train_labels

## Training

Initialize the weight of the linear regression

In [ ]:
n_features = train_data.shape[1]
linear_weights = 0 # TODO, initialize a random tensor 

Training loop

In [ ]:
num_iterations = 512
lr = 1e-3
for i in range(num_iterations):
    # TODO

Get the real predictions

In [ ]:
predictions = (predictions * trains_std) + trains_mean
predictions

Our predictions seem very close to the ground truth!

**To go further**: Stochastic Gradient Descent is not the optimal algorithm in terms of convergeance.
If you are curious, you can read this nice article about an improvement to SGD, momentum and try to implement it: https://distill.pub/2017/momentum/